In [6]:
import pandas as pd
import numpy as np

from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error


In [7]:
train_filename_iq = ( './../datasets/dengue_test_iq.pkl' )
dengue_test_iq = pd.read_pickle( train_filename_iq )

test_filename_sj = ( './../datasets/dengue_test_sj.pkl' )
dengue_test_sj = pd.read_pickle( test_filename_sj )

submission_sj = pd.DataFrame( dengue_test_sj, columns = ['city'])
submission_iq = pd.DataFrame( dengue_test_iq, columns = ['city'])
submission_sj.reset_index( inplace = True)
submission_sj['year'] = submission_sj['week_start_date'].dt.year
submission_sj['weekofyear'] = submission_sj['week_start_date'].dt.weekofyear
submission_iq.reset_index( inplace = True)
submission_iq['year'] = submission_iq['week_start_date'].dt.year
submission_iq['weekofyear'] = submission_iq['week_start_date'].dt.weekofyear
submission = submission_sj.append( submission_iq )
submission.drop( columns = ['week_start_date'], inplace = True )
sub_sj = pd.DataFrame()
sub_iq = pd.DataFrame()

In [16]:
for month in range( 1, 13 ):
    train_filename = ( './../datasets/train_sj_month_' + str( month ) + '.pkl' )
    test_filename  = ( './../datasets/test_sj_month_' + str( month ) + '.pkl' )
    dengue_train_sj_month = pd.read_pickle( train_filename )
    dengue_test_sj_month  = pd.read_pickle( test_filename )
    X = pd.DataFrame( dengue_train_sj_month )
    X.drop( columns = ['city','year','total_cases','month'], inplace = True )
    y = dengue_train_sj_month.total_cases
    X_sj_Full_train, X_sj_Full_test, Y_sj_Full_train, Y_sj_Full_test = train_test_split(
                                                                    X, y, shuffle = False)
    param_grid = { 
        "n_estimators"      : [160,220,150],
        "max_features"      : ["sqrt"],
        "min_samples_split" : [10,12,18],
        "bootstrap"         : [True],
        "max_depth"         : [2,4,5,8]
        }
    estimator = RandomForestRegressor( criterion='mae', oob_score=True)
    rf_est = GridSearchCV(estimator, param_grid,
                          n_jobs=-1, cv=5, verbose=0,
                          iid=True ).fit( X_sj_Full_train, Y_sj_Full_train)
    Y_sj_test_pred = rf_est.best_estimator_.predict(X_sj_Full_test).astype(int)
    print ("San Juan Test  MAE error :", mean_absolute_error(Y_sj_test_pred, Y_sj_Full_test))
    Y_sj_train_pred = rf_est.best_estimator_.predict(X_sj_Full_train).astype(int)
    print ("San Juan Train MAE error :", mean_absolute_error(Y_sj_train_pred, Y_sj_Full_train))
    X_test = pd.DataFrame( dengue_test_sj_month, columns = X.columns )
    submit_pred_sj = pd.DataFrame( dengue_test_sj_month, columns = ['city','week_start_date'])
    submit_pred_sj['total_cases'] = rf_est.best_estimator_.predict(X_test).astype( 'int')
    sub_sj = sub_sj.append( submit_pred_sj, sort = False )
sub_sj = sub_sj.drop( columns=['week_start_date'])
sub_sj.reset_index( inplace = True )
sub_sj['year'] = sub_sj['week_start_date'].dt.year
sub_sj['weekofyear'] = sub_sj['week_start_date'].dt.weekofyear
sub_sj.drop( columns = ['week_start_date'], inplace = True )

San Juan Test  MAE error : 17.782608695652176
San Juan Train MAE error : 12.537313432835822
San Juan Test  MAE error : 13.166666666666666
San Juan Train MAE error : 8.74074074074074
San Juan Test  MAE error : 8.222222222222221
San Juan Train MAE error : 6.574074074074074
San Juan Test  MAE error : 7.260869565217392
San Juan Train MAE error : 4.924242424242424
San Juan Test  MAE error : 6.944444444444445
San Juan Train MAE error : 5.403846153846154
San Juan Test  MAE error : 10.222222222222221
San Juan Train MAE error : 7.314814814814815
San Juan Test  MAE error : 28.82608695652174
San Juan Train MAE error : 15.462686567164178
San Juan Test  MAE error : 33.388888888888886
San Juan Train MAE error : 23.25925925925926
San Juan Test  MAE error : 37.8421052631579
San Juan Train MAE error : 22.31578947368421
San Juan Test  MAE error : 38.04545454545455
San Juan Train MAE error : 28.0
San Juan Test  MAE error : 36.111111111111114
San Juan Train MAE error : 26.85185185185185
San Juan Test  MAE

ValueError: cannot insert level_0, already exists

In [ ]:
for month in range( 1, 13 ):
    train_filename = ( './../datasets/train_iq_month_' + str( month ) + '.pkl' )
    test_filename  = ( './../datasets/test_iq_month_' + str( month ) + '.pkl' )
    dengue_train_iq_month = pd.read_pickle( train_filename )
    dengue_test_iq_month  = pd.read_pickle( test_filename )
    X = pd.DataFrame( dengue_train_iq_month )
    X.drop( columns = ['city','year','total_cases','month'], inplace = True )
    y = dengue_train_iq_month.total_cases
    X_iq_train, X_iq_test, Y_iq_train, Y_iq_test = train_test_split( X, y, shuffle = False)
    param_grid = { 
        "n_estimators"      : [160,220,100],
        "max_features"      : ["sqrt"],
        "min_samples_split" : [10,16,20],
        "bootstrap"         : [True],
        "max_depth"         : [2,4,5,8]
        }
    estimator = RandomForestRegressor( criterion='mae', oob_score=True)
    rf_est = GridSearchCV(estimator, param_grid,
                          n_jobs=-1, cv=5, verbose=0, iid=True).fit( X_iq_train, Y_iq_train )
    Y_iq_pred = rf_est.best_estimator_.predict(X_iq_test).astype(int)
    print ("Iquitos Test  MAE error :", mean_absolute_error(Y_iq_pred, Y_iq_test))
    pred_train_iq = rf_est.best_estimator_.predict(X_iq_train).astype(int)
    print ("Iquitos Train MAE error :", mean_absolute_error(pred_train_iq, Y_iq_train))
    X_test = pd.DataFrame( dengue_test_iq_month, columns = X.columns )
    submit_pred_iq = pd.DataFrame( dengue_test_iq_month, columns = ['city', 'week_start_date'])
    submit_pred_iq['total_cases'] = rf_est.best_estimator_.predict(X_test).astype( 'int')
    sub_iq = sub_iq.append( submit_pred_iq, sort = False )
sub_iq.drop( columns=['week_start_date'], inplace = True)
sub_iq.reset_index( inplace = True )
sub_iq['year'] = sub_iq['week_start_date'].dt.year
sub_iq['weekofyear'] = sub_iq['week_start_date'].dt.weekofyear
sub_iq.drop( columns = ['week_start_date'], inplace = True )

In [ ]:
submit = sub_sj.append( sub_iq, sort=True )

In [ ]:
submission_RF = pd.merge( submission, submit, on=['city','year','weekofyear'], how='left' )

In [ ]:
submission_RF.columns = ['city','year','weekofyear','total_cases']
submission_RF.to_excel("data/submission_month_RF.xlsx", index = False)

In [ ]:
submission_RF.to_csv("data/submission_month_RF.csv", index = False)